In [2]:
import numpy as np

class Oracle:

    def value(self, x: np.ndarray) -> float:

        raise NotImplementedError()
        
    def gradient(self, x: np.ndarray) -> np.ndarray:

        raise NotImplementedError()

In [39]:
import numpy as np

class Adam:
    '''представляет Adam оптимизацию

    Fields:
        eta: скорость обучения
        beta1: оценка первого момента 
        beta2: оценка второго момента
        epsilon: сглаж. коэф
    '''

    eta: float
    beta1: float
    beta2: float
    epsilon: float

    def __init__(self, *, eta: float = 0.1, beta1: float = 0.9, beta2: float = 0.999, epsilon: float = 1e-8):
        '''Инициализация `eta`, `gamma`, and `epsilon` fields'''
        self.eta = eta          # Скорость обучения
        self.beta1              #  первого момента 
        self.beta2              # оценка второго момента
        self.epsilon = epsilon  # сглаживающий коэф.
        self.vector = 0         # сумма корней градиента
        self.m = 0          # First moment vector
        self.v = 0           # Second moment vector
        self.t = 0              # Time step

    def optimize(self, oracle: Oracle, x0: np.ndarray, *,
                 max_iter: int = 100, eps: float = 1e-5) -> np.ndarray:
        '''Оптимизирует функцию со стартовой точки х0.
        Оптимизация заканчивается когда мы достигли макс. числа итераций
        или если наша точка оказалась ниже порога

        Аргументы функции:
            oracle: функция которую нужно оптимизировать
            x0: стартовая точка спуска
            max_iter: максимальное число итераций (ограничение)
            eps: граница

        Возвращает:
            Точку остановки
        '''

        x = x0
        self.vector = np.zeros_like(x0)
        self.m = np.zeros_like(x0)
        self.v = np.zeros_like(x0)

        for _ in range(max_iter):
            self.t += 1
            gradient = oracle.gradient(x) # вычисление градиента в текущей точке
            if np.linalg.norm(gradient) < eps:
                # проверка на пересечение порога
                break

            self.vector = self.gamma * self.vector + (1-self.gamma) * (np.power(gradient,2))  # обновление вектора
            
            adjusted_grad = (self.eta / (np.sqrt(self.vector + self.epsilon))) * gradient
            x -= adjusted_grad
        
        return x

In [40]:
# Example usage of Oracle and RMSProp
class QuadraticOracle(Oracle):
    '''Example quadratic function for optimization'''

    def value(self, x: np.ndarray) -> float:
        return np.sum(x ** 2)  # Function value f(x) = x^2
    
    def gradient(self, x: np.ndarray) -> np.ndarray:
        return 2 * x  # Gradient f'(x) = 2x


# Using RMSProp to optimize a quadratic function
oracle = QuadraticOracle()
initial_point = np.array([10.0, -5.0])  # Initial point
optimizer = RMSProp(eta=0.1)

optimal_point = optimizer.optimize(oracle, initial_point)
print("Оптимальная точка:", optimal_point)


Оптимальная точка: [ 8.75865195e-01 -2.37220548e-16]
